In [133]:
import pandas as pd
import numpy as np
from datetime import datetime


In [134]:
K_PROJECT = 'Project'
K_ACCOUNT = 'Account'
K_PAYMENT_ACCOUNT = 'Payment account'
K_MERCHANT = 'Merchant'
K_ADDRESS = 'Address'
K_NOTE = 'Note'
K_TAGS = 'Tags'
K_AUTHOR = 'Author'
K_IMAGE1 = 'Image 1'
K_IMAGE2 = 'Image 2'
K_IMAGE3 = 'Image 3'
K_DATETIME = 'Date time'
K_TYPE = 'Type'
K_CATEGORY = 'Category'
K_EXPENSE = 'Expense'
K_INCOME = 'Income'
K_TRANSFER = 'Transfer'
K_INVESTING = 'Investing'
K_MONTH = 'Month'
K_AMOUNT = 'Amount'


In [135]:
def parseDatetime(x: str):
    rus = {"янв": "jan",
           "февр": "feb", "фев": "feb",
           "мар": "mar",
           "апр": "apr",
           "май": "may", "мая": "may",
           "июн": "jun",
           "июл": "jul",
           "авг": "aug",
           "сент": "sep", "сен": "sep",
           "окт": "oct",
           "нояб": "nov", "ноя": "nov",
           "дек": "dec",
           "г. ": "", 
           ".": ""}
    for r, e in rus.items():
        x = x.lower().replace(r, e)
    try:
        dt = datetime.strptime(x.lower(), u'%d %b %Y %H:%M:%S')
    except:
        dt = datetime.strptime(x.lower(), u'%d%m%Y %H:%M:%S')
    return dt


In [136]:
df = pd.read_csv("data/test.csv", on_bad_lines="warn", sep="\t")
df = df.drop(columns=[K_PROJECT, K_ACCOUNT, K_PAYMENT_ACCOUNT, K_MERCHANT,
                      K_ADDRESS, K_NOTE, K_TAGS, K_AUTHOR, 
                      K_IMAGE1, K_IMAGE2, K_IMAGE3])
df[K_DATETIME] = df[K_DATETIME].apply(lambda x: parseDatetime(x))
df[K_DATETIME] = pd.to_datetime(df[K_DATETIME]).astype(np.int64)
df[K_DATETIME] = df[K_DATETIME].apply(lambda x: x/1000000000)
df[K_AMOUNT] = df[K_AMOUNT].apply(lambda x: x.replace(u"\u00A0", '').replace(',','.'))
df[K_AMOUNT] = df[K_AMOUNT].astype(float)


In [137]:
start_date = datetime.fromtimestamp(df[K_DATETIME].min())
end_date = datetime.fromtimestamp(df[K_DATETIME].max())

month_list = pd.period_range(start=start_date, end=end_date, freq='M')
month_list = [month.strftime("%m-%Y") for month in month_list]


In [138]:
expense = df[df[K_TYPE] == K_EXPENSE]
income = df[df[K_TYPE] == K_INCOME]
invest = df[(df[K_TYPE] == K_TRANSFER) & (df[K_CATEGORY] == K_INVESTING)]



In [139]:
def transformByCat(data:pd.DataFrame):
    unique_cats = data[K_CATEGORY].unique()
    data_by_cat = pd.DataFrame(columns=unique_cats)
    data_by_cat[K_MONTH] = month_list
    data_by_cat = data_by_cat.set_index(K_MONTH)

    for col in data_by_cat.columns:
        data_by_cat[col].values[:] = 0

    for index, row in data.iterrows():
        date = datetime.fromtimestamp(row[K_DATETIME])
        month = date.strftime("%m-%Y")
        cat = row[K_CATEGORY]
        value = row[K_AMOUNT]
        data_by_cat.loc[data_by_cat.index == month, cat] += abs(value)

    return data_by_cat

